In [1]:
import json
import itertools
import random
import pandas as pd

In [99]:
def clean(text: str):
    clean_text = text.replace("Я с радостью помогу вам сгенерировать поле «О себе» для вашего резюме. У меня получилось придумать такой текст: ", "")
    clean_text = clean_text.replace("Я с радостью помогу вам сгенерировать поле «О себе» для вашего резюме. Учтите, что в этом поле нужно описать себя так, чтобы работодатель захотел вас нанять. Вот что у меня получилось: ", "")
    if clean_text[0] == '"':
        clean_text = clean_text[1:]
    if clean_text[-1] == '"':
        clean_text = clean_text[:-2]
    return clean_text

In [100]:
def fix_prompt(text: str):
    fixed_text = text.replace("Водительское удостоверение: отсутствует", "Водительское удостоверение: не указано")
    return fixed_text

In [101]:
def all_possible_combs(good: list, bad: list):
    return list(itertools.product(good, bad))

In [102]:
def random_combs(good: list, bad: list):
    good_set = set(good)
    bad_set = set(bad)
    combs = []
    while good_set and bad_set:
        elem_good = random.choice(tuple(good_set))
        elem_bad = random.choice(tuple(bad_set))
        combs.append((elem_good, elem_bad))
        good_set.remove(elem_good)
        bad_set.remove(elem_bad)
    return combs

### preprocess dataset

In [103]:
with open("./data/example_dpo_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [104]:
res = {}
for item in data:
    prompt = item["data"]["prompt"]
    res[prompt] = {}
    res[prompt]["good"] = []
    res[prompt]["bad"] = []

    good_ids = item["annotations"][0]['result'][0]["value"]["ranker"]["relevant_results"]
    bad_ids = item["annotations"][0]['result'][0]["value"]["ranker"]["biased_results"]

    items = item["data"]["items"]
    for lbl_item in items:
        if lbl_item["id"] in good_ids:
            res[prompt]["good"].append(lbl_item["body"])
        elif lbl_item["id"] in bad_ids:
            res[prompt]["bad"].append(lbl_item["body"])


In [ ]:
res

In [106]:
all_combs_data = []
random_combs_data = []

for prompt, answers in res.items():
    fixed_prompt = fix_prompt(prompt)
    good = answers["good"]
    clean_good = [clean(text) for text in good]
    bad = answers["bad"]

    all_combs = all_possible_combs(clean_good, bad) # list of tuples
    for g, b in all_combs:
        all_combs_data.append({"prompt": fixed_prompt,
                               "chosen": g,
                               "rejected": b})
    rand_combs = random_combs(clean_good, bad)
    for g, b in rand_combs:
        random_combs_data.append({"prompt": fixed_prompt,
                                  "chosen": g,
                                  "rejected": b})

In [107]:
len(random_combs_data)

1188

In [108]:
len(all_combs_data)

6295

In [109]:
df_all = pd.DataFrame(data=all_combs_data)

In [110]:
df_all.head()

,prompt,chosen,rejected
0,Пол: мужчина. Возраст: 38 лет. Желаемая зарпла...,Я идеальный кандидат на должность администрати...,Я с радостью помогу вам сгенерировать поле «О ...
1,Пол: мужчина. Возраст: 38 лет. Желаемая зарпла...,Я идеальный кандидат на должность администрати...,Я с радостью помогу вам сгенерировать поле «О ...
2,Пол: мужчина. Возраст: 38 лет. Желаемая зарпла...,Я идеальный кандидат на должность администрати...,"""Я идеальный кандидат для должности Администра..."
3,Пол: мужчина. Возраст: 38 лет. Желаемая зарпла...,"Мужчина, 47 лет, проживаю в Москве. Обладаю во...",Я с радостью помогу вам сгенерировать поле «О ...
4,Пол: мужчина. Возраст: 38 лет. Желаемая зарпла...,"Мужчина, 47 лет, проживаю в Москве. Обладаю во...",Я с радостью помогу вам сгенерировать поле «О ...


In [111]:
df_all.to_csv("./data/all_pos_combs_dpo_data.csv")

In [112]:
df_rnd = pd.DataFrame(data=random_combs_data)

In [113]:
df_rnd.head()

,prompt,chosen,rejected
0,Пол: мужчина. Возраст: 38 лет. Желаемая зарпла...,Я идеальный кандидат на должность администрати...,Я с радостью помогу вам сгенерировать поле «О ...
1,Пол: мужчина. Возраст: 38 лет. Желаемая зарпла...,"Мужчина, 47 лет, проживаю в Москве. Обладаю во...",Я с радостью помогу вам сгенерировать поле «О ...
2,Пол: мужчина. Возраст: 38 лет. Желаемая зарпла...,Я опытный менеджер по продажам с водительским ...,"""Я идеальный кандидат для должности Администра..."
3,Пол: мужчина. Возраст: 18 лет. Желаемая зарпла...,Я молодой и энергичный оператор с опытом работ...,"Я мужчина, проживаю в Москве. Имею опыт работы..."
4,Пол: мужчина. Возраст: 18 лет. Желаемая зарпла...,"Я мужчина, проживаю в Москве и готов к новым в...",Я с радостью помогу вам сгенерировать поле «О ...


In [114]:
df_rnd.to_csv("./data/random_combs_dpo_data.csv")

### split

In [115]:
TRAIN_RATIO = 0.9

In [116]:
def get_train_eval(df, train_ratio: float):
    """ создает train и eval датасеты указанной пропорции"""                                                  
                                                    
    num_rows = int(len(df) * train_ratio)
    
    train_df = df[:num_rows]
    eval_df = df[num_rows:]
    
    return train_df, eval_df    

In [117]:
train_df_all, eval_df_all = get_train_eval(df_all, TRAIN_RATIO)
train_df_rnd, eval_df_rnd = get_train_eval(df_rnd, TRAIN_RATIO)

In [118]:
train_df_all.to_json(f"./data/all/train_all.json", orient="records", lines=True)
eval_df_all.to_json(f"./data/all/eval_all.json", orient="records", lines=True)

train_df_rnd.to_json(f"./data/rnd/train_rnd.json", orient="records", lines=True)
eval_df_rnd.to_json(f"./data/rnd/eval_rnd.json", orient="records", lines=True)